# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [1]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [2]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i])/2
        end
    end

    return x_tilde
end
    



init_x0_lsq (generic function with 1 method)

In [3]:
function Compute_Alpha_dual(v_k, dir_v_k)
    n = length(v_k)
    if n == 0
        return 0.
    end
    alphas = ones(n)
    for i=1:n
        if dir_v_k[i] < 0
            alphas[i] = -v_k[i] * 0.999 / dir_v_k[i]
        end
    end
    return minimum(alphas)
end


    
function Compute_Alpha_primal(v_k, dir_v_k, lvar, uvar)
    n = length(v_k)
    alpha_l, alpha_u = ones(n), ones(n)
    for i=1:n
        if dir_v_k[i] > 0
            alpha_u[i] = (uvar[i] - v_k[i]) * 0.999 / dir_v_k[i]
        elseif dir_v_k[i] < 0
            alpha_l[i] = (lvar[i] - v_k[i]) * 0.999 / dir_v_k[i]
        end
    end
    return min(minimum(alpha_l), minimum(alpha_u))
end

function Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s_l' * (x_l-lvar) + s_u' * (uvar-x_u)) / (length(x_l) + length(x_u))
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [34]:
function MehrotraPCQuadBounds(QM, max_iter; eps=1e-6, tol_step_x=1e-6, eps_mu=1e-6, display=true)
    
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    x_k = init_x0(lvar, uvar)
    id_non_inf_l, id_non_inf_u = findall((x -> x!=-Inf), lvar), findall((x -> x!=Inf), uvar) # finite bounds index
    @assert all(x_k .> lvar) && all(x_k .< uvar)
    A = jac(QM, x_k)
    n_rows, n_cols = size(A) 
    Q = hess(QM, x_k)
    c = grad(QM, zeros(n_cols))
    c0 = obj(QM, zeros(n_cols))
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    x_k = init_x0_lsq(A, b, lvar, uvar)
    @assert all(x_k .> lvar) && all(x_k .< uvar)
    s_k_l, s_k_u = zeros(n_cols), zeros(n_cols)
    s_k_l[id_non_inf_l] .= 1.
    s_k_u[id_non_inf_u] .= 1.
    lambda_k = Matrix(A)'\(c +Q*x_k) # least square initialisation, s_0 = stilde_0
    
    rb_k = A * x_k - b
    rc_k = -Q * x_k + A' * lambda_k + s_k_l - s_k_u - c
    mu_k = Compute_mu(x_k[id_non_inf_l], x_k[id_non_inf_u], 
                        s_k_l[id_non_inf_l], s_k_u[id_non_inf_u], 
                        lvar[id_non_inf_l], uvar[id_non_inf_u])

    k = 0
    e = ones(n_cols)
    n_c = norm(c)
    n_b = norm(b)
    
    # matrices without infinity constraints
    nb_non_inf_l, nb_non_inf_u = length(id_non_inf_l), length(id_non_inf_u) # number of finite constraints
    
    Xk_non_inf_l = Diagonal(x_k[id_non_inf_l])
    Sk_non_inf_l = spzeros(nb_non_inf_l, n_cols)
    I_non_inf_l = spzeros(n_cols, nb_non_inf_l)
    for j=1:nb_non_inf_l   # cartesian index not working when nb_non_inf_l = 0
        Sk_non_inf_l[j, id_non_inf_l[j]] = s_k_l[id_non_inf_l[j]]
        I_non_inf_l[id_non_inf_l[j], j] = 1
    end
    
    Xk_non_inf_u = Diagonal(x_k[id_non_inf_u])
    Sk_non_inf_u = spzeros(nb_non_inf_u, n_cols)
    I_op_non_inf_u = spzeros(n_cols, nb_non_inf_u)
    for j=1:nb_non_inf_u 
        Sk_non_inf_u[j, id_non_inf_u[j]] = s_k_u[id_non_inf_u[j]]
        I_op_non_inf_u[id_non_inf_u[j], j] = -1
    end
    Lvar_non_inf = Diagonal(lvar[id_non_inf_l]) 
    Uvar_non_inf = Diagonal(uvar[id_non_inf_u]) 

    # stopping criterion
    quad_part = x_k' * Q * x_k 
    pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2))
    cond_rb, cond_rc = norm(rb_k) / (1 + n_b), norm(rc_k) / (1 + n_c)
    opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps # start false?
    small_step_x, small_mu = false, mu_k < eps_mu

    # display
    if display == true
        println("Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu")
        println("--------------------------------------------------------------------------------------------------------")
        @printf("% 4d |    % 7.4e   |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,0., mu_k)
    end
    

    while k<max_iter && (!opti_pdd || !opti_rb || !opti_rc) && !small_step_x && !small_mu
        
        
        # Affine scaling direction
        Jacob_Fk = [-Q            A'                             I_non_inf_l                         I_op_non_inf_u
                    A             spzeros(n_rows, n_rows)        spzeros(n_rows, nb_non_inf_l)       spzeros(n_rows, nb_non_inf_u)
                    Sk_non_inf_l  spzeros(nb_non_inf_l, n_rows)  Xk_non_inf_l-Lvar_non_inf           spzeros(nb_non_inf_l, nb_non_inf_u)
                    Sk_non_inf_u  spzeros(nb_non_inf_u, n_rows)  spzeros(nb_non_inf_u,nb_non_inf_l)  Xk_non_inf_u-Uvar_non_inf]
        Fk_aff = [-rc_k
                  -rb_k
                  -(x_k[id_non_inf_l]-lvar[id_non_inf_l]).*s_k_l[id_non_inf_l]
                  -(x_k[id_non_inf_u]-uvar[id_non_inf_u]).*s_k_u[id_non_inf_u]]

        dir_aff_k = Jacob_Fk\Fk_aff
        
        alpha_aff_pri = Compute_Alpha_primal(x_k, dir_aff_k[1:n_cols], lvar, uvar)
        alpha_aff_dual_l = Compute_Alpha_dual(s_k_l[id_non_inf_l], 
                                            dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        alpha_aff_dual_u = Compute_Alpha_dual(s_k_u[id_non_inf_u], 
                                                dir_aff_k[n_rows+n_cols+nb_non_inf_l+1:end])
        # pb above compute alpha dual

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        if nb_non_inf_l == 0
            alpha_aff_dual_final = alpha_aff_dual_u
        elseif nb_non_inf_u == 0
            alpha_aff_dual_final = alpha_aff_dual_l
        else
            alpha_aff_dual_final = min(alpha_aff_dual_l, alpha_aff_dual_u)
        end
        
        mu_aff = Compute_mu(x_k[id_non_inf_l] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_l],
                    x_k[id_non_inf_u] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_u],
                    s_k_l[id_non_inf_l] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l],
                    s_k_u[id_non_inf_u] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end],
                    lvar[id_non_inf_l], uvar[id_non_inf_u])
        
        sigma = (mu_aff / mu_k)^3

        # corrector and centering step
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e[1:nb_non_inf_l] - dir_aff_k[1:n_cols][id_non_inf_l].*dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
                 -sigma*mu_k*e[1:nb_non_inf_u] - dir_aff_k[1:n_cols][id_non_inf_u].*dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end]] 

        dir_cc_k = Jacob_Fk\Fk_cc
        dir_k = dir_aff_k + dir_cc_k # final direction
        
        alpha_k_pri = Compute_Alpha_primal(x_k, dir_k[1:n_cols], lvar, uvar)
        alpha_k_dual_l = Compute_Alpha_dual(s_k_l[id_non_inf_l], 
                                        dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        
        alpha_k_dual_u = Compute_Alpha_dual(s_k_u[id_non_inf_u], 
                                            dir_k[n_rows+n_cols+nb_non_inf_l+1: end])
        
        if nb_non_inf_l == 0
            alpha_dual_final = alpha_k_dual_u
        elseif nb_non_inf_u == 0
            alpha_dual_final = alpha_k_dual_l
        else
            alpha_dual_final = min(alpha_k_dual_l, alpha_k_dual_u)
        end
        
        # new parameters
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_dual_final * dir_k[n_cols+1: n_rows+n_cols]
        s_k_l[id_non_inf_l] += alpha_dual_final * dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
        s_k_u[id_non_inf_u] += alpha_dual_final * dir_k[n_rows+n_cols+nb_non_inf_l+1: end]
        
        Xk_non_inf_l = Diagonal(x_k[id_non_inf_l])
        for j=1:nb_non_inf_l
            Sk_non_inf_l[j, id_non_inf_l[j]] = s_k_l[id_non_inf_l[j]]
        end
        Xk_non_inf_u = Diagonal(x_k[id_non_inf_u])
        
        for j=1:nb_non_inf_u
            Sk_non_inf_u[j, id_non_inf_u[j]] = s_k_u[id_non_inf_u[j]]
        end
        
        step_x = norm(alpha_k_pri * dir_k[1:n_cols])
        mu_k = Compute_mu(x_k[id_non_inf_l], x_k[id_non_inf_u], 
                        s_k_l[id_non_inf_l], s_k_u[id_non_inf_u], 
                        lvar[id_non_inf_l], uvar[id_non_inf_u])

        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k_l - s_k_u - c
        
        # update stopping criterion values:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k - s_k_l[id_non_inf_l]'*lvar[id_non_inf_l] +  s_k_u[id_non_inf_u]'*uvar[id_non_inf_u] ) / 
                    (1 + abs(c' * x_k + quad_part/2)) 
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
        small_step_x, small_mu = step_x < tol_step_x, mu_k < eps_mu
        k += 1
        
        if display == true
            @printf("% 4d |    % 7.4e   |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,step_x, mu_k)
        end
        

    end
    
    if display == true
        criteria = [k >= max_iter,  opti_pdd, opti_rb, opti_rc, small_step_x, small_mu]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", 
            "step_x <= small_step_x", "mu_k <= eps_mu"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    elapsed_time = time() - start_time
    
    if k>= max_iter
        status = :max_iter
    else
        status = :acceptable
    end
    
    GenExStats = GenericExecutionStats(status, QM, solution = x_k,
                               objective = c0 + c'*x_k + x_k'*Q*x_k/2 , 
                                dual_feas = norm(rc_k), primal_feas = norm(rb_k),
                                multipliers = lambda_k,
                                multipliers_L = s_k_l,
                                multipliers_U = s_k_u,
                                iter = k, elapsed_time=elapsed_time)
    return GenExStats
end


MehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [28]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [29]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [30]:
stats_mpc1 =  MehrotraPCQuadBounds(SM, 20)
println(stats_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.4000e+01   |         6.67e-01       |    5.59e-01  |    2.37e-01  |  0.00e+00  |  1.67e+00
   1 |     1.2894e+01   |         6.89e-03       |    2.96e-02  |    4.17e-02  |  1.34e+00  |  3.25e-02
   2 |     1.3501e+01   |         1.31e-03       |    6.02e-04  |    8.48e-04  |  1.18e-01  |  2.00e-03
   3 |     1.3500e+01   |         1.23e-06       |    6.26e-07  |    8.30e-07  |  2.27e-03  |  2.03e-06
   4 |     1.3500e+01   |         1.23e-09       |    6.26e-10  |    8.30e-10  |  2.25e-06  |  2.03e-09

 stopping criterion = ["pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", "mu_k <= eps_mu"]
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.500000000893674
  primal feasibility: 2.504818221598968e-9
  dual feasibility: 8.348990

In [31]:
n = 1
df_test = DataFrame(id = Int[],
         name = String[],
         status = Symbol[],
         objective = Float64[],
         time = Float64[],
         iter = Int[],
         primal_feas = Float64[],
         dual_feas = Float64[])

push!(df_test, Dict(:id => 1,
         :name => "test" ,
         :status => stats_mpc1.status,
         :objective => stats_mpc1.objective,
         :time => stats_mpc1.elapsed_time,
         :iter => 12,
         :primal_feas => stats_mpc1.primal_feas,
         :dual_feas => stats_mpc1.dual_feas))

push!(df_test, Dict(:id => 2,
         :name => "test2" ,
         :status => stats_mpc1.status,
         :objective => stats_mpc1.objective,
         :time => stats_mpc1.elapsed_time,
         :iter => 11,
         :primal_feas => stats_mpc1.primal_feas,
         :dual_feas => stats_mpc1.dual_feas))

df_test
markdown_table(stdout, df_test)

|    id |  name |     status | objective |     time |  iter | primal_feas | du ⋯
|-------|-------|------------|-----------|----------|-------|-------------|--- ⋯
|     1 |  test | acceptable |   1.4e+01 |  2.0e-01 |    12 |     2.5e-09 |    ⋯
|     2 | test2 | acceptable |   1.4e+01 |  2.0e-01 |    11 |     2.5e-09 |    ⋯


In [9]:
using DataFrames, Printf, Random

Random.seed!(0)

n = 10
names = [:alpha, :beta, :gamma]
stats = Dict(name => DataFrame(:id => 1:n,
         :name => [@sprintf("prob%03d", i) for i = 1:n],
         :status => map(x -> x < 0.75 ? :success : :failure, rand(n)),
         :f => randn(n),
         :t => 1e-3 .+ rand(n) * 1000,
         :iter => rand(10:10:100, n),
         :irrelevant => randn(n)) for name in names)
markdown_table(stdout, stats[:alpha])

|    id |    name |  status |        f |        t |  iter | irrelevant |
|-------|---------|---------|----------|----------|-------|------------|
|     1 | prob001 | failure | -6.9e-01 |  6.2e+01 |    70 |    8.9e-01 |
|     2 | prob002 | failure | -7.6e-01 |  3.5e+02 |    10 |   -2.3e+00 |
|     3 | prob003 | success |  4.0e-01 |  7.7e+02 |    10 |   -1.2e+00 |
|     4 | prob004 | success |  8.1e-01 |  4.3e+01 |    80 |   -8.8e-01 |
|     5 | prob005 | success | -3.5e-01 |  2.7e+02 |    30 |   -4.9e-01 |
|     6 | prob006 | success | -1.9e-01 |  6.7e+01 |    80 |    5.1e-01 |
|     7 | prob007 | success | -1.6e+00 |  1.6e+02 |    60 |   -7.6e-01 |
|     8 | prob008 | success | -2.5e+00 |  6.1e+02 |    40 |   -7.7e-01 |
|     9 | prob009 | success |  2.3e+00 |  1.4e+02 |    40 |   -3.0e-02 |
|    10 | prob010 | failure |  2.2e-01 |  8.4e+02 |    50 |   -1.3e+00 |


In [10]:
stats_mpc1.elapsed_time

5.0959999561309814

In [11]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
stats_mpc_verif =  MehrotraPCQuadBounds(SM_verif, 20);
println(stats_mpc_verif)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.4000e+01   |         6.67e-01       |    5.59e-01  |    2.37e-01  |  0.00e+00  |  1.67e+00
   1 |     1.2894e+01   |         6.89e-03       |    2.96e-02  |    4.17e-02  |  1.34e+00  |  3.25e-02
   2 |     1.3501e+01   |         1.31e-03       |    6.02e-04  |    8.48e-04  |  1.18e-01  |  2.00e-03
   3 |     1.3500e+01   |         1.23e-06       |    6.26e-07  |    8.30e-07  |  2.27e-03  |  2.03e-06
   4 |     1.3500e+01   |         1.23e-09       |    6.26e-10  |    8.30e-10  |  2.25e-06  |  2.03e-09

 stopping criterion = ["pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", "mu_k <= eps_mu"]
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.500000000893674
  primal feasibility: 2.504818221598968e-9
  dual feasibility: 8.348990

In [12]:
### pb verif 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
stats_mpc_verif2 =  MehrotraPCQuadBounds(SM_verif2, 20);
println(stats_mpc_verif2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.0000e+00   |         3.33e-01       |    1.00e+00  |    9.57e-01  |  0.00e+00  |  1.00e+00
   1 |     1.1111e-01   |         4.04e-01       |    0.00e+00  |    1.84e-16  |  1.19e+00  |  1.50e-01
   2 |     6.4513e-03   |         6.75e-03       |    0.00e+00  |    5.55e-17  |  1.28e-01  |  2.26e-03
   3 |     6.4513e-06   |         6.79e-06       |    0.00e+00  |    3.83e-20  |  7.89e-03  |  2.26e-06
   4 |     6.4513e-09   |         6.79e-09       |    0.00e+00  |    1.11e-16  |  7.89e-06  |  2.26e-09

 stopping criterion = ["pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", "mu_k <= eps_mu"]
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 6.451292182287191e-9
  primal feasibility: 0.0
  dual feasibility: 2.2204460492504393e-16


# Lecture des donnees .SIF

In [13]:

function createQuadraticModel(qpdata)
    # probleme du point initial
    x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, x0=x0)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

In [14]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [15]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
#displayQuadraticModel(SM2)

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [16]:
stats_mpc2 =  MehrotraPCQuadBounds(SM2, 1000, tol_step_x = 1e-10);
println(stats_mpc2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     4.3820e+02   |         6.83e-01       |    4.44e-01  |    9.54e-01  |  0.00e+00  |  3.64e+01
   1 |     3.4744e+02   |         5.55e+00       |    4.22e-01  |    9.22e-01  |  1.45e+01  |  3.61e+01
   2 |    -5.4715e+00   |         2.38e+02       |    3.85e-01  |    8.58e-01  |  6.16e+01  |  3.54e+01
   3 |     3.6743e+01   |         3.61e+01       |    3.84e-16  |    7.04e-01  |  2.20e+02  |  3.31e+01
   4 |     6.7364e+01   |         1.07e+01       |    4.28e-16  |    3.31e-01  |  4.45e+01  |  1.76e+01
   5 |    -1.5940e+02   |         1.70e+00       |    5.84e-16  |    1.83e-01  |  3.13e+02  |  9.31e+00
   6 |    -4.0241e+02   |         3.17e-01       |    2.29e-15  |    4.03e-02  |  4.98e+02  |  3.99e+00
   7 |    -4.5934e+02   |         4.20e-02       |    1.03e-15  | 

In [17]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
SM3 = SlackModel(QM3);

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


In [18]:
stats_mpc3 =  MehrotraPCQuadBounds(SM3, 20, tol_step_x = 1e-12);
println(stats_mpc3)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.1675e+01   |         9.21e-01       |    2.49e+03  |    4.63e-01  |  0.00e+00  |  6.18e+00
   1 |     2.4577e+01   |         1.05e+01       |    2.08e+03  |    2.52e-01  |  2.93e+00  |  4.63e+00
   2 |     9.7097e-01   |         5.71e+01       |    4.40e+02  |    1.11e-01  |  6.36e+00  |  1.86e+00
   3 |    -6.3645e+00   |         9.90e+00       |    2.04e+02  |    7.76e-02  |  2.74e+01  |  1.37e+00
   4 |    -3.0768e+01   |         7.38e-01       |    7.82e+01  |    5.18e-02  |  1.26e+02  |  1.04e+00
   5 |    -3.3028e+02   |         8.64e-01       |    5.41e-13  |    4.01e-02  |  1.54e+03  |  1.47e+00
   6 |    -5.2433e+02   |         8.69e-01       |    7.05e-13  |    3.82e-02  |  9.83e+02  |  1.80e+00
   7 |    -6.2643e+02   |         8.27e-01       |    1.03e-12  | 

In [19]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path_pb, "\\SC50A.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4)
SM4 = SlackModel(QM4);

┌ Info: Using 'SC50A' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'MAXIM' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'CONST' as RHS (l. 153)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [20]:
stats_mpc4 =  MehrotraPCQuadBounds(SM4, 20, tol_step_x = 1e-12);
println(stats_mpc4)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |    -1.0000e+00   |         5.00e-01       |    1.30e+01  |    4.42e+00  |  0.00e+00  |  2.01e+01
   1 |    -8.1774e-01   |         5.84e+02       |    1.30e-02  |    3.06e+00  |  3.55e+01  |  1.45e+01
   2 |    -8.9022e-01   |         2.36e+01       |    1.30e-05  |    1.30e-01  |  7.66e-01  |  6.12e-01
   3 |    -9.3339e+00   |         3.42e+00       |    1.04e-05  |    1.23e-01  |  1.11e+02  |  5.93e-01
   4 |    -3.2552e+01   |         4.92e-01       |    7.80e-06  |    7.91e-02  |  3.01e+02  |  4.57e-01
   5 |    -5.4487e+01   |         1.03e-02       |    2.91e-06  |    6.01e-02  |  2.99e+02  |  3.24e-01
   6 |    -6.0759e+01   |         2.04e-02       |    8.31e-07  |    2.89e-02  |  1.03e+02  |  1.96e-01
   7 |    -6.3542e+01   |         1.94e-02       |    1.64e-07  | 

# Tests sur tous les problèmes

In [38]:
df_test = DataFrame(id = Int[],
         name = String[],
         status = Symbol[],
         objective = Float64[],
         time = Float64[],
         iter = Int[],
         primal_feas = Float64[],
         dual_feas = Float64[])



i = 0
i_stop = 11   #number of problems to solve
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   &&# problems with fixed variables
                                        !(file_name in ["25FV47.SIF"; "BNL1.SIF";
                                                        "BRANDY.SIF"; "CRE-A.SIF"]) # singular error
                            
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end

        SM_i = SlackModel(createQuadraticModel(qpdata_i))

        stats_mpc_i =  MehrotraPCQuadBounds(SM_i, 40, display=false)
        
        # 
        push!(df_test, Dict(:id => i,
                             :name => file_name ,
                             :status => stats_mpc_i.status,
                             :objective => stats_mpc_i.objective,
                             :time => stats_mpc_i.elapsed_time,
                             :iter => stats_mpc_i.iter,
                             :primal_feas => stats_mpc_i.primal_feas,
                             :dual_feas => stats_mpc_i.dual_feas))
    end
    
    if i == i_stop
        break
        break
    end
end

markdown_table(stdout, df_test)

ADLITTLE.SIF


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350


AFIRO.SIF
AGG.SIF


┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'AGG' as RHS (l. 1852)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


AGG2.SIF


...

┌ Info: Using 'BOEING1' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 23)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS1' as RHS (l. 2372)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGE1' as RANGES (l. 2446)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552


BOEING2.SIF


┌ Info: Using 'INTBOU' as BOUNDS (l. 2492)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622
┌ Info: Using 'BOEING2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 23)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS1' as RHS (l. 898)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'RANGE1' as RANGES (l. 919)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:552


|    id |         name |     status | objective |     time |  iter | primal_fe ⋯
|-------|--------------|------------|-----------|----------|-------|---------- ⋯
|     1 | ADLITTLE.SIF | acceptable |   2.3e+05 |  8.8e-02 |    38 |     9.8e- ⋯
|     2 |    AFIRO.SIF | acceptable |  -4.6e+02 |  1.8e-02 |    10 |     8.3e- ⋯
|     3 |      AGG.SIF |   max_iter |   1.9e+09 |  6.4e-01 |    40 |     3.2e+ ⋯
|     4 |     AGG2.SIF |   max_iter |  -4.3e+06 |  1.2e+00 |    40 |     4.9e+ ⋯
|     5 |     AGG3.SIF |   max_iter |   1.7e+09 |  1.2e+00 |    40 |     1.6e+ ⋯
|     6 |    BANDM.SIF | acceptable |  -1.6e+02 |  2.9e-01 |    21 |     3.1e- ⋯
|     7 | BEACONFD.SIF | acceptable |   3.4e+04 |  1.5e-01 |    10 |     7.3e- ⋯
|     8 |    BLEND.SIF | acceptable |  -3.1e+01 |  3.2e-02 |    11 |     4.6e- ⋯
|     9 |     BNL2.SIF |   max_iter |   1.8e+03 |  1.4e+01 |    40 |     1.2e- ⋯
|    10 |  BOEING1.SIF |   max_iter |  -3.3e+02 |  1.1e+00 |    40 |     1.4e- ⋯
|    11 |  BOEING2.SIF | acc

┌ Info: Using 'INTBOU' as BOUNDS (l. 930)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


In [57]:
T = typeof(stats_mpc1)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname status is Symbol
type of the fieldname solution is Array{T,1} where T
type of the fieldname objective is Real
type of the fieldname dual_feas is Real
type of the fieldname primal_feas is Real
type of the fieldname multipliers is Array{T,1} where T
type of the fieldname multipliers_L is Array{T,1} where T
type of the fieldname multipliers_U is Array{T,1} where T
type of the fieldname iter is Int64
type of the fieldname counters is NLSCounters
type of the fieldname elapsed_time is Real
type of the fieldname solver_specific is Dict{Symbol,Any}


In [59]:
# obj  9.8723216072E+05
qpdata5 = readqps(string(path_pb, "\\25FV47.SIF"))
QM5 = createQuadraticModel(qpdata5)
SM5 = SlackModel(QM5);

┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [60]:
stats_mpc_5 =  MehrotraPCQuadBounds(SM5, 20)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     6.0144e+04   |         3.29e-01       |    5.78e+01  |    5.98e-01  |  0.00e+00  |  1.16e+02


SingularException: SingularException(0)

In [40]:
SM5.meta


  Problem name: Generic-slack
   All variables: ████████████████████ 1876   All constraints: ████████████████████ 821   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████████⋅⋅⋅ 1571             lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 305              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 821   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 821   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.30% sparsity)   10705 



In [41]:
SM2.meta

  Problem name: Generic-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   

